In [1]:
!wget -nc https://lazyprogrammer.me/course_files/spam.csv

--2025-01-30 13:51:06--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.02s   

2025-01-30 13:51:08 (22.1 MB/s) - ‘spam.csv’ saved [503663/503663]



In [2]:
from  tensorflow.keras.layers import LSTM,Dense,Embedding,InputLayer,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv("spam.csv",encoding="ISO-8859-1")

In [4]:
data.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
data=data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)

In [6]:
data.columns=["label","data"]
data

,label,data
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
for i in range(len(data)):
  if data["label"][i]=="ham":
    data["label"][i]=0
  else:
    data["label"][i]=1
data

<ipython-input-7-31d50e7a9fab>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["label"][i]=0
<ipython-input-7-31d50e7a9fab>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through c

,label,data
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [8]:
x=data["data"]
y=data["label"].values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [10]:
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [11]:
tokenizers=Tokenizer(num_words=20000)
tokenizers.fit_on_texts(X_train)

x_train=tokenizers.texts_to_sequences(X_train)
x_test=tokenizers.texts_to_sequences(X_test)

n=len(tokenizers.word_index)


In [12]:
x_train=pad_sequences(x_train)
max_length=x_train.shape[1]
x_test=pad_sequences(x_test,maxlen=max_length)


In [13]:
model=Sequential([
                  InputLayer(input_shape=(max_length,)),
                  Embedding(n+1,64),
                  LSTM(50,activation='relu'),
                  Dropout(0.2),
                  Dense(50,activation='relu'),
                  Dense(1,activation="sigmoid")
                  ]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [14]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 80ms/step - accuracy: 0.8640 - loss: 8874.0000 - val_accuracy: 0.8655 - val_loss: 0.4146
Epoch 2/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - accuracy: 0.8858 - loss: 252.0231 - val_accuracy: 0.9722 - val_loss: 0.1360
Epoch 3/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 72ms/step - accuracy: 0.9812 - loss: 0.0980 - val_accuracy: 0.9803 - val_loss: 0.0810
Epoch 4/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.9891 - loss: 0.0466 - val_accuracy: 0.9830 - val_loss: 0.0644
Epoch 5/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 68ms/step - accuracy: 0.9934 - loss: 0.0271 - val_accuracy: 0.9857 - val_loss: 0.0571
Epoch 6/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 11s 72ms/step - accuracy: 0.9954 - loss: 0.0195 - val_accuracy: 0.9865 - val_loss: 0.0549
Epoch 7/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 10s 73ms/step - accuracy: 0.9977 - loss: 0.0139 - val_accuracy: 0.9857 - val_loss: 0.0555
Epoch 8/20
140/140 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.9987 - loss: 0.0109

In [15]:
model.save("hello.h5")